In [157]:
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

In [158]:
# Import, process, and combine the data from multiple JSON files

json_files = ['../1-100.json', '../101-300.json', '../750-999.json']

def process_data(data, source_file_name):

    id_to_item = {item['Id']: item for item in data}
    
    parent_to_all_children = {}
    for item in data:
        if 'Relationships' in item:
            for relationship in item['Relationships']:
                # Initialize a dictionary for the item if it doesn't exist
                if item['Id'] not in parent_to_all_children:
                    parent_to_all_children[item['Id']] = {}
                # Append the child IDs under the appropriate relationship type
                parent_to_all_children[item['Id']].setdefault(relationship['Type'], []).extend(relationship['Ids'])
    
    cell_records = []

    for item in data:
        if item.get('BlockType') == 'TABLE':
            table_id = item['Id']
            relationships = parent_to_all_children.get(table_id, {})
            
            # Initialize an empty list to hold cell records, including cell_type
            aggregated_cells = []
            for rel_type in ['CHILD', 'MERGED_CELL', 'TABLE_FOOTER', 'TABLE_TITLE']:
                cell_ids = relationships.get(rel_type, [])
                for cell_id in cell_ids:
                    # Append both cell_id and its relationship type to the list
                    aggregated_cells.append((cell_id, rel_type))
            
            for cell_id, cell_type in aggregated_cells:
                cell_block = id_to_item.get(cell_id)
                if not cell_block:
                    continue

                entity_type = cell_block.get('EntityTypes', [None])[0] if 'EntityTypes' in cell_block else None

                cell_geometry = cell_block['Geometry']['BoundingBox']
                
                child_ids = parent_to_all_children.get(cell_id, {}).get('CHILD', [])
                cell_words = [id_to_item[child_id]['Text'] for child_id in child_ids if child_id in id_to_item and 'Text' in id_to_item[child_id]]
                cell_content = ' '.join(cell_words)
                
                cell_records.append({
                    'cell_id': cell_id,
                    'cell_type': cell_type,  # Include the cell_type here
                    'entity_type': entity_type,
                    'cell_words': cell_words,
                    'cell_content': cell_content,
                    'cell_width': cell_geometry['Width'],
                    'cell_height': cell_geometry['Height'],
                    'cell_left': cell_geometry['Left'],
                    'cell_top': cell_geometry['Top'],
                    'row_index': cell_block.get('RowIndex', None),
                    'column_index': cell_block.get('ColumnIndex', None),
                    'row_span': cell_block.get('RowSpan', 1),
                    'column_span': cell_block.get('ColumnSpan', 1),
                    'table_id': table_id,
                    'table_type': item['EntityTypes'][0] if 'EntityTypes' in item else None,
                    'table_width': item['Geometry']['BoundingBox']['Width'],
                    'table_height': item['Geometry']['BoundingBox']['Height'],
                    'table_left': item['Geometry']['BoundingBox']['Left'],
                    'table_top': item['Geometry']['BoundingBox']['Top'],
                    'table_page': item['Page'],
                    'source': source_file_name,
                    # Additional fields can be added here
                })
    return pd.DataFrame(cell_records)

# Process each file and combine the results as before
cells_dfs = [process_data(data, file_path.split('/')[-1]) for file_path in json_files for data in [json.load(open(file_path))]]
cells_dfs = pd.concat(cells_dfs, ignore_index=True)

# Display the length of the dataframe
len(cells_dfs)


96132

In [159]:
# Establish meaningful cell content for each cell by examining merged cell relationships


# Ensure the necessary columns are in the correct data type
cells_dfs['row_index'] = cells_dfs['row_index'].fillna(0).astype(int)
cells_dfs['column_index'] = cells_dfs['column_index'].fillna(0).astype(int)
cells_dfs['row_span'] = cells_dfs['row_span'].fillna(1).astype(int)  # Default span of 1 if missing
cells_dfs['column_span'] = cells_dfs['column_span'].fillna(1).astype(int)

# Sort the DataFrame as required
cells_dfs.sort_values(by=['table_id', 'column_index', 'row_index'], inplace=True)

# Isolate merged cells
merged_cells = cells_dfs[cells_dfs['cell_type'] == 'MERGED_CELL']

# Initialize a column for tracking merged cell parent ID and merge status
cells_dfs['merged_parent_cell_id'] = np.nan
cells_dfs['has_merged_parent'] = 0

for cell in merged_cells.itertuples():
    # Calculate the affected range of rows and columns
    affected_rows = range(cell.row_index, cell.row_index + cell.row_span)
    affected_columns = range(cell.column_index, cell.column_index + cell.column_span)

    # Find the cells that are affected
    affected_cells = cells_dfs[
        (cells_dfs['table_id'] == cell.table_id) &
        (cells_dfs['cell_type'] == 'CHILD') &  # Targeting only child cells
        (cells_dfs['row_index'].isin(affected_rows)) &
        (cells_dfs['column_index'].isin(affected_columns))
    ]

    # Aggregate text content of affected cells, stripping to remove leading/trailing spaces
    aggregated_text_content = " ".join(filter(None, affected_cells['cell_content'].astype(str))).strip()

    if aggregated_text_content:
        # Update the affected cells with the aggregated text content and merge-related information
        cells_dfs.loc[affected_cells.index, 'cell_content'] = aggregated_text_content
        cells_dfs.loc[affected_cells.index, 'merged_parent_cell_id'] = cell.cell_id
        cells_dfs.loc[affected_cells.index, 'has_merged_parent'] = 1

# Fill missing values for new columns
cells_dfs['has_merged_parent'] = cells_dfs['has_merged_parent'].fillna(0).astype(int)
# Do not convert merged_parent_cell_id to int; leave it as is or ensure it's treated as a string/object
cells_dfs['merged_parent_cell_id'] = cells_dfs['merged_parent_cell_id'].fillna('None')

# Optional: If you want to ensure 'merged_parent_cell_id' is explicitly recognized as a string/object column:
cells_dfs['merged_parent_cell_id'] = cells_dfs['merged_parent_cell_id'].astype(str)

# Display the length of the DataFrame
len(cells_dfs)


96132

In [160]:
# Summarize cell contents and child entities for each table in tables_df_without_titles


# Function to aggregate cell contents into a list of lists, one per row
def aggregate_contents(group):
    # Sort the group by row and column index to ensure the correct order
    sorted_group = group.sort_values(by=['row_index', 'column_index'])
    # Aggregate contents by row
    contents_by_row = sorted_group.groupby('row_index')['cell_words'].apply(list).tolist()
    return contents_by_row

def aggregate_child_entities(group):
    # Filter the group to only include CHILD cells
    child_cells = group[group['cell_type'] == 'CHILD']
    # Replace NaN or empty entity_type values with 'normal'
    child_cells['entity_type'] = child_cells['entity_type'].replace({np.nan: 'normal', '': 'normal'})
    # Sort the group by row and column index to ensure the correct order
    sorted_group = child_cells.sort_values(by=['row_index', 'column_index'])
    # Aggregate entity types by row
    entities_by_row = sorted_group.groupby('row_index')['entity_type'].apply(list).tolist()
    return entities_by_row

# Aggregate information for each table
tables_df_without_titles = cells_dfs.groupby('table_id').apply(lambda g: pd.Series({
    'table_width': g['table_width'].max(),
    'table_height': g['table_height'].max(),
    'table_left': g['table_left'].max(),
    'table_top': g['table_top'].max(),
    'table_page': g['table_page'].max(),
    'source': g['source'].iloc[0],
    'cell_count': g['cell_words'].count(),
    'row_count': int(g['row_index'].max()),
    'column_count': int(g['column_index'].max()),
    'content': aggregate_contents(g),
    'entities': aggregate_child_entities(g),
    # Add counts for different cell types
    'child_count': g[g['cell_type'] == 'CHILD']['cell_type'].count(),
    'merged_cell_count': g[g['cell_type'] == 'MERGED_CELL']['cell_type'].count(),
    'table_title_count': g[g['cell_type'] == 'TABLE_TITLE']['cell_type'].count(),
    'table_footer_count': g[g['cell_type'] == 'TABLE_FOOTER']['cell_type'].count(),
    'table_type': g['table_type'].max()
})).reset_index()


# Calculate page title per page of each document based on confidence scores, store in titles_df, and merge with tables_df_without_titles as tables_df


def process_layout_titles(data, source_file_name):
    layout_title_ids = [item['Id'] for item in data if item.get('BlockType') == 'LAYOUT_TITLE']
    id_to_item = {item['Id']: item for item in data}
    
    layout_titles = []
    for layout_title in layout_title_ids:
        layout_title_block = id_to_item[layout_title]
        layout_title_cell = {
            'layout_title_id': layout_title,
            'layout_title_text': ' '.join([id_to_item[child_id]['Text'] for child_id in layout_title_block.get('Relationships', [{}])[0].get('Ids', []) if child_id in id_to_item and 'Text' in id_to_item[child_id]]),
            'layout_title_page': layout_title_block['Page'],
            'layout_title_confidence': layout_title_block['Confidence'],
            'source': source_file_name,  # Keep track of the source document
        }
        layout_titles.append(layout_title_cell)

    doc_titles_df = pd.DataFrame(layout_titles)
    
    # Perform calculations within the current document's scope
    doc_titles_df['max_confidence_per_page'] = doc_titles_df.groupby('layout_title_page')['layout_title_confidence'].transform('max')
    doc_titles_df['is_max_confidence'] = doc_titles_df['layout_title_confidence'] == doc_titles_df['max_confidence_per_page']
    doc_titles_df.drop(columns=['max_confidence_per_page'], inplace=True)
    
    return doc_titles_df

# Initialize an empty list to hold DataFrames from all files
titles_df = []

# Process each JSON file separately and append the results to the list
for file_path in json_files:
    with open(file_path) as file:
        data = json.load(file)
        doc_titles_df = process_layout_titles(data, file_path.split('/')[-1])
        titles_df.append(doc_titles_df)

# Concatenate all DataFrames after processing
titles_df = pd.concat(titles_df, ignore_index=True)

tables_df = pd.merge(tables_df_without_titles, titles_df[titles_df['is_max_confidence'] == True][['source', 'layout_title_page', 'layout_title_text']],
                     left_on=['source', 'table_page'], right_on=['source', 'layout_title_page'], how='left')

tables_df.drop(columns=['layout_title_page'], inplace=True, errors='ignore')

print(len(tables_df))

2703


In [161]:
# Preprocessing steps for the table content and entities


## ONE-HOT ENCODING FOR TABLE_TYPE
one_hot_encoder = OneHotEncoder()

table_type_encoded = one_hot_encoder.fit_transform(tables_df[['table_type']])
table_type_encoded_dense = table_type_encoded.toarray()
column_names = one_hot_encoder.get_feature_names_out(['table_type'])
table_type_encoded_df = pd.DataFrame(table_type_encoded_dense, columns=column_names)

tables_df = pd.concat([tables_df.reset_index(drop=True), table_type_encoded_df.reset_index(drop=True)], axis=1)

print('Completed one-hot encoding for table_type:', len(tables_df))

Completed one-hot encoding for table_type: 2703


In [163]:
from joblib import load

tfidf_vectorizer_content = load('../corpuses//tfidf_vectorizer_content.joblib')
tfidf_vectorizer_titles = load('../corpuses/tfidf_vectorizer_titles.joblib')

In [164]:
new_content_corpus = tables_df['content'].apply(lambda x: ' '.join(map(str, x)))
new_titles_corpus = tables_df['layout_title_text'].apply(lambda x: str(x))  # Assuming this is already a string or similar operation if needed

In [165]:
tfidf_layout_title_features = tfidf_vectorizer_titles.transform(new_titles_corpus)
tfidf_content_features = tfidf_vectorizer_content.transform(new_content_corpus)

In [166]:
# For content features
content_feature_names = [f'content_tfidf_{i}' for i in range(tfidf_content_features.shape[1])]
tfidf_content_df = pd.DataFrame(tfidf_content_features.toarray(), columns=content_feature_names)

# For title features
title_feature_names = [f'title_tfidf_{i}' for i in range(tfidf_layout_title_features.shape[1])]
tfidf_layout_title_df = pd.DataFrame(tfidf_layout_title_features.toarray(), columns=title_feature_names)

In [168]:
# Reset index on the original DataFrame if necessary to ensure alignment
tables_df.reset_index(drop=True, inplace=True)

# Concatenate the original DataFrame with the new TF-IDF DataFrames
tables_df = pd.concat([tables_df, tfidf_content_df, tfidf_layout_title_df], axis=1)


In [169]:
## LABEL ENCODING FOR ENTITIES
def pad_all_lists(list_of_list_of_lists, pad_value=0):
    # Determine the longest inner list length
    max_inner_length = max(len(inner) for outer in list_of_list_of_lists for inner in outer)
    
    # Determine the max number of inner lists within any outer list
    max_outer_length = max(len(outer) for outer in list_of_list_of_lists)
    
    # Pad inner lists
    padded_inner = [[inner + [pad_value] * (max_inner_length - len(inner)) for inner in outer] for outer in list_of_list_of_lists]
    
    # Pad outer lists to ensure they all have the same number of inner lists, with each inner list padded to the same length
    padded_outer = [outer + [[pad_value] * max_inner_length] * (max_outer_length - len(outer)) for outer in padded_inner]
    
    return padded_outer


# Extract all unique entity types
unique_entity_types = set(entity for sublist in tables_df['entities'] for item in sublist for entity in item)

# Map unique entity types to integers starting from 1
entity_type_to_int = {entity: i + 1 for i, entity in enumerate(sorted(unique_entity_types))}

# Transform the entities lists using the mapping
tables_df['entities_int'] = tables_df['entities'].apply(lambda x: [[entity_type_to_int[entity] for entity in sublist] for sublist in x])

print("\nUnique entity types:", unique_entity_types)
print("Entity type to integer mapping:", entity_type_to_int)

# Apply padding to the 'entities_int' column
tables_df['entities_int_padded'] = pad_all_lists(tables_df['entities_int'])
tables_df.drop('entities_int', axis=1, inplace=True) 

print(f'\nMax table rows observed in training dataset: {tables_df["row_count"].max()}')
print(f'Max table columns observed in training dataset: {tables_df["column_count"].max()}')
print(f'\nPadded all entities lists to contain {len(tables_df["entities_int_padded"][0])} rows and {len(tables_df["entities_int_padded"][0][0])} columns')

## Flatten the entities_int_padded column
flattened_entities = np.array([np.array(row).flatten() for row in tables_df['entities_int_padded']])

print(f"\nNew shape of entities feature: {flattened_entities.shape}")



Unique entity types: {'normal', 'TABLE_SECTION_TITLE', 'COLUMN_HEADER', 'TABLE_FOOTER', 'TABLE_TITLE'}
Entity type to integer mapping: {'COLUMN_HEADER': 1, 'TABLE_FOOTER': 2, 'TABLE_SECTION_TITLE': 3, 'TABLE_TITLE': 4, 'normal': 5}

Max table rows observed in training dataset: 16
Max table columns observed in training dataset: 6

Padded all entities lists to contain 16 rows and 6 columns

New shape of entities feature: (2703, 96)


In [170]:
tables_df.to_pickle('tables_df.pkl')

In [171]:
tables_df.to_csv('tables_df.csv')

## Following are for further processing cells

In [42]:
## NOTE: THIS TAKES AROUND 27.5 MINUTES TO RUN FOR 100,000 ROWS

# Sort the DataFrame as required
combined_cells_df.sort_values(by=['table_id', 'column_index', 'row_index'], inplace=True)

# Initialize empty lists to store the 'words' in each direction for each cell
words_above_list = []
words_left_list = []
words_below_list = []
words_right_list = []

# Filter the DataFrame to only include rows where cell_type is 'CHILD'
child_cells_df = combined_cells_df[combined_cells_df['cell_type'] == 'CHILD']

# Iterate over rows of child_cells_df instead of the entire cells_df
for index, row in child_cells_df.iterrows():
    # Adjust masks to include only CHILD cells for comparison
    above_mask = (combined_cells_df['table_id'] == row['table_id']) & \
                 (combined_cells_df['column_index'] == row['column_index']) & \
                 (combined_cells_df['row_index'] < row['row_index']) & \
                 (combined_cells_df['cell_type'] == 'CHILD')
    above_cells = combined_cells_df.loc[above_mask]

    left_mask = (combined_cells_df['table_id'] == row['table_id']) & \
                (combined_cells_df['row_index'] == row['row_index']) & \
                (combined_cells_df['column_index'] < row['column_index']) & \
                (combined_cells_df['cell_type'] == 'CHILD')
    left_cells = combined_cells_df.loc[left_mask]

    below_mask = (combined_cells_df['table_id'] == row['table_id']) & \
                 (combined_cells_df['column_index'] == row['column_index']) & \
                 (combined_cells_df['row_index'] > row['row_index']) & \
                 (combined_cells_df['cell_type'] == 'CHILD')
    below_cells = combined_cells_df.loc[below_mask]

    right_mask = (combined_cells_df['table_id'] == row['table_id']) & \
                 (combined_cells_df['row_index'] == row['row_index']) & \
                 (combined_cells_df['column_index'] > row['column_index']) & \
                 (combined_cells_df['cell_type'] == 'CHILD')
    right_cells = combined_cells_df.loc[right_mask]
    
    # Process each direction's cells to aggregate words, replacing None with 'empty' and joining words within a cell
    words_above = [' '.join([word if word is not None else 'empty' for word in cell_words]) if cell_words else 'empty' for cell_words in above_cells['cell_words']]
    words_left = [' '.join([word if word is not None else 'empty' for word in cell_words]) if cell_words else 'empty' for cell_words in left_cells['cell_words']]
    words_below = [' '.join([word if word is not None else 'empty' for word in cell_words]) if cell_words else 'empty' for cell_words in below_cells['cell_words']]
    words_right = [' '.join([word if word is not None else 'empty' for word in cell_words]) if cell_words else 'empty' for cell_words in right_cells['cell_words']]

    # Append the list to the respective direction list
    words_above_list.append(words_above)
    words_left_list.append(words_left)
    words_below_list.append(words_below)
    words_right_list.append(words_right)

# Since we're iterating over child_cells_df, we need to merge the results back into the original DataFrame
# Create a temporary DataFrame with the results
temp_df = pd.DataFrame({
    'index': child_cells_df.index,
    'words_above': words_above_list,
    'words_left': words_left_list,
    'words_below': words_below_list,
    'words_right': words_right_list
})

# Merge the temporary DataFrame back into the original DataFrame based on the index
combined_cells_df = pd.merge(combined_cells_df, temp_df, how='left', left_index=True, right_on='index')

# Drop the 'index' column as it's no longer needed
combined_cells_df.drop(columns=['index'], inplace=True)

combined_cells_df.head()

,cell_id,cell_type,cell_words,cell_content,cell_width,cell_height,cell_left,cell_top,row_index,column_index,row_span,column_span,table_id,source,merged_parent_cell_id,has_merged_parent,words_above,words_left,words_below,words_right
NaN,428073c6-299d-4272-bbd3-d0ac3bfa59b6,TABLE_TITLE,"[Trip, Unit, Settings]",Trip Unit Settings,0.845572,0.021403,0.086389,0.463036,0,0,1,1,0003dc62-ff11-4a22-b5c4-e2c9a9de8137,750-999.json,None,0,NaN,NaN,NaN,NaN
0.0,d8f19ff7-799f-4a91-af81-e9fb623cd012,CHILD,"[Trip, Unit, Settings]",Trip Unit Settings,0.221201,0.021520,0.087440,0.462512,1,1,1,1,0003dc62-ff11-4a22-b5c4-e2c9a9de8137,750-999.json,3794ae51-54bd-43ca-90ab-79c08b479a2d,1,[],[],"[Element, Long Time Element Settings, Long Tim...","[empty, empty, empty, empty]"
NaN,3794ae51-54bd-43ca-90ab-79c08b479a2d,MERGED_CELL,[],,0.845572,0.021962,0.087640,0.462069,1,1,1,5,0003dc62-ff11-4a22-b5c4-e2c9a9de8137,750-999.json,None,0,NaN,NaN,NaN,NaN
1.0,db6d13c1-2282-4577-91c6-eeb11fdda4c1,CHILD,[Element],Element,0.221002,0.019851,0.087650,0.483867,2,1,1,1,0003dc62-ff11-4a22-b5c4-e2c9a9de8137,750-999.json,None,0,[Trip Unit Settings],[],"[Long Time Element Settings, Long Time Pick Up...","[Ranges, As Found, As Left, As Tested]"
2.0,a441f1f5-2720-4118-9dc8-95493f9d62b8,CHILD,"[Long, Time, Element, Settings]",Long Time Element Settings,0.221202,0.019851,0.087461,0.503555,3,1,1,1,0003dc62-ff11-4a22-b5c4-e2c9a9de8137,750-999.json,b36b94e0-1c4d-48e3-8021-d2bd1975a088,1,"[Trip Unit Settings, Element]",[],"[Long Time Pick Up:, Long Time Delay:, Short T...","[empty, empty, empty, empty]"


In [43]:
combined_cells_df.to_csv('combined_cells3.csv', index=False)

## This is probably not applicable anymore

In [147]:
## TF-IDF VECTORIZATION FOR MEANINGFUL STRINGS

# Create content_string column from content column
tables_df['content_string'] = tables_df['content'].apply(lambda x: ' '.join(map(str, x)))

# Initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')

# Fit and transform the text fields to TF-IDF weighted matrices
tfidf_content = tfidf_vectorizer.fit_transform(tables_df['content_string'])
tfidf_layout_title_text = tfidf_vectorizer.fit_transform(tables_df['layout_title_text'])

# Convert the TF-IDF matricies to dense arrays
tfidf_content = tfidf_content.toarray()
tfidf_layout_title_text = tfidf_layout_title_text.toarray()

tfidf_content_df = pd.DataFrame(tfidf_content, columns=[f'content_tfidf_{i+1}' for i in range(tfidf_content.shape[1])])
tfidf_layout_title_text_df = pd.DataFrame(tfidf_layout_title_text, columns=[f'layout_title_tfidf_{i+1}' for i in range(tfidf_layout_title_text.shape[1])])
tables_df.reset_index(drop=True, inplace=True)
tables_df = pd.concat([tables_df, tfidf_content_df, tfidf_layout_title_text_df], axis=1)

# Check the shape of the resulting TF-IDF matrix
print(f"\nNew shape of content_string feature: {tfidf_content.shape}")
print(f"New shape of layout_title_text feature: {tfidf_layout_title_text.shape}")


New shape of content_string feature: (2703, 1000)
New shape of layout_title_text feature: (2703, 16)
